In [1]:
from Assessment.Credibility_assessment import run_credibility_assessment
from Assessment.Performance_assessment import run_performance_assessment
from Assessment.run_assessment import prepare_data_and_model, test_get_stocks_recommendation
from Assessment.utils import parse_args, normalize_assessment_results_list

D:\ProgramFiles\Miniconda\envs\qlib\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\ProgramFiles\Miniconda\envs\qlib\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\ProgramFiles\Miniconda\envs\qlib\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import argparse
from Assessment.utils import ParseConfigFile
import sys
def parse_args():
    parser = argparse.ArgumentParser()

    # model
    parser.add_argument('--model_name', default='relation_GATs')
    parser.add_argument('--model_path', default='D:\Research\Fintech\K-Quant\parameter')
    parser.add_argument('--num_relation', type= int, default=134)
    parser.add_argument('--d_feat', type=int, default=6)
    parser.add_argument('--hidden_size', type=int, default=128)
    parser.add_argument('--num_layers', type=int, default=2)
    parser.add_argument('--dropout', type=float, default=0.1)
    parser.add_argument('--K', type=int, default=1)
    parser.add_argument('--loss_type', default='')
    parser.add_argument('--config', action=ParseConfigFile, default='')
    # for ts lib model
    parser.add_argument('--seq_len', type=int, default=60)
    parser.add_argument('--moving_avg', type=int, default=21)
    parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
    parser.add_argument('--embed', type=str, default='timeF',
                        help='time features encoding, options:[timeF, fixed, learned]')
    parser.add_argument('--freq', type=str, default='b',
                        help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
    parser.add_argument('--distil', action='store_false',
                        help='whether to use distilling in encoder, using this argument means not using distilling',
                        default=False)
    parser.add_argument('--factor', type=int, default=1, help='attn factor')
    parser.add_argument('--n_heads', type=int, default=1, help='num of heads')
    parser.add_argument('--d_ff', type=int, default=64, help='dimension of fcn')
    parser.add_argument('--activation', type=str, default='gelu', help='activation')
    parser.add_argument('--e_layers', type=int, default=8, help='num of encoder layers')
    parser.add_argument('--top_k', type=int, default=5, help='for TimesBlock')
    parser.add_argument('--pred_len', type=int, default=-1, help='the length of pred squence, in regression set to -1')
    parser.add_argument('--de_norm', default=True, help='de normalize or not')

    # data
    parser.add_argument('--data_set', type=str, default='csi360')
    parser.add_argument('--target', type=str, default='t+0')
    parser.add_argument('--pin_memory', action='store_false', default=True)
    parser.add_argument('--batch_size', type=int, default=-1)  # -1 indicate daily batch
    parser.add_argument('--least_samples_num', type=float, default=1137.0)
    parser.add_argument('--label', default='')  # specify other labels
    parser.add_argument('--start_date', default='2019-01-01')
    parser.add_argument('--end_date', default='2019-01-05')

    # input for csi 300
    parser.add_argument('--data_root', default='D:\Research\Fintech\K-Quant\Data')
    parser.add_argument('--market_value_path', default= 'D:\Research\Fintech\K-Quant\Data\csi300_market_value_07to20.pkl')

    parser.add_argument('--stock2concept_matrix', default='D:\ProjectCodes\K-Quant\Data\csi300_stock2concept.npy')


    parser.add_argument('--stock2stock_matrix', default='D:\Research\Fintech\K-Quant\Data\csi300_multi_stock2stock_hidy_2024.npy')
    parser.add_argument('--stock_index', default='D:\Research\Fintech\K-Quant\Data\csi300_stock_index_2024.npy')
    parser.add_argument('--model_dir', default='D:\Research\Fintech\K-Quant\parameter')
    parser.add_argument('--overwrite', action='store_true', default=False)
    parser.add_argument('--device', default='cpu')
    filtered_args = [arg for arg in sys.argv if not arg.startswith('-f') and not arg.endswith('.json')]
    args = parser.parse_args(filtered_args[1:])  # Skip the script name
    return args




In [3]:
args = parse_args()

model_list = ['LSTM', 'GRU', 'MLP', 'NRSR', 'relation_GATs']
# model_list = ['NRSR']
explanation_model = "inputGradientExplainer"
seq_len_list = [30, 60]

args.start_date = '2022-06-01'
args.end_date = '2022-06-02'

args.seq_len = 60
c_a_r_list = []
for seq_len in seq_len_list:
    for model in model_list:
        h_p_dict = {
            "prediction_model": model,
            "explanation_model": explanation_model,
            "start_date": args.start_date,
            "end_date": args.end_date,
            "seq_len": seq_len
        }
        args.model_name = model
        args.seq_len = seq_len

        data_loader, param_dict, model = prepare_data_and_model(args)
        credibility_assessment_results_dict = run_credibility_assessment(param_dict, data_loader, model,
                                                                         explanation_model)

        c_a_r_list.append((h_p_dict, credibility_assessment_results_dict))

n_c_a_r_list = normalize_assessment_results_list(c_a_r_list, num_selection = 5)

print(n_c_a_r_list)

predict in  LSTM


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 71.67it/s]


predict in  GRU


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 14.86it/s]


predict in  MLP


...

predict in  NRSR


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.02it/s]


predict in  relation_GATs


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.70it/s]

[({'prediction_model': 'LSTM', 'explanation_model': 'inputGradientExplainer', 'start_date': '2022-06-01', 'end_date': '2022-06-02', 'seq_len': 30}, {'可靠性得分': 0.01315918269616674, '稳定性得分': 0.58189999539838, '鲁棒性得分': 0.2862492457317245, '透明性得分': 0.0, '解释效果得分': 0.0}), ({'prediction_model': 'GRU', 'explanation_model': 'inputGradientExplainer', 'start_date': '2022-06-01', 'end_date': '2022-06-02', 'seq_len': 30}, {'可靠性得分': 0.0, '稳定性得分': 0.6537007410284782, '鲁棒性得分': 0.0, '透明性得分': 0.0, '解释效果得分': 0.0}), ({'prediction_model': 'MLP', 'explanation_model': 'inputGradientExplainer', 'start_date': '2022-06-01', 'end_date': '2022-06-02', 'seq_len': 30}, {'可靠性得分': 0.9999999999999998, '稳定性得分': 0.0, '鲁棒性得分': 1.0, '透明性得分': 0.0, '解释效果得分': 0.0}), ({'prediction_model': 'NRSR', 'explanation_model': 'inputGradientExplainer', 'start_date': '2022-06-01', 'end_date': '2022-06-02', 'seq_len': 30}, {'可靠性得分': 0.024895915330210447, '稳定性得分': 0.9999999999999999, '鲁棒性得分': 0.19046097834299047, '透明性得分': 1.0, '解释效果得分': 0.

In [4]:
args.model_name = "NRSR"
explanation_model = "inputGradientExplainer"
args.seq_len = 60
args.num_recommendation_stocks = 3
data_loader, param_dict, model = prepare_data_and_model(args)
recommend_stocks_list = test_get_stocks_recommendation(param_dict, data_loader, model,
                                                       top_n=args.num_recommendation_stocks)  # 输出的是推荐的股票

print(recommend_stocks_list)

predict in  NRSR


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 11.80it/s]

['300413.SZ', '603882.SH', '300122.SZ']


In [6]:
select_dict_list = [
        {
            '002460.SZ': 3,

        },
        {
            '600009.SH': 1,

        },
        {
            '600000.SH': 100,

        },
        {
            '600015.SH': 1,
        },
        {
            '600703.SH': 1,
        },
        {
            '300072.SZ': 1
        },
    ]
args.start_date = args.start_date.replace('-', '')
args.end_date = args.end_date.replace('-', '')
args.return_preference = 0 # 输入回报偏好
args.risk_preference = 90 # 输入风险偏好
p_a_r_list = []
for select_dict in select_dict_list:
    h_p_dict = {
                    "select_dict": select_dict,
                    "return_preference": args.return_preference,
                    "seq_len": args.risk_preference,
                    "start_date": args.start_date,
                    "end_date": args.end_date,

                }
    performance_assessment_results_dict = run_performance_assessment(args, select_dict) # 输出性能得分
    print(performance_assessment_results_dict)
    p_a_r_list.append((h_p_dict, performance_assessment_results_dict))

    n_p_a_r_list = normalize_assessment_results_list(p_a_r_list, num_selection=5)
    print(n_p_a_r_list)

{'002460.SZ': 3}
002460.SZ 能源金属 7


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  8.17it/s]


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
002460.SZ: 近1天年化收益率位于同行业股票Top100%(7/7), 年化波动率位于同行业股票Top100% (7/7)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 8.838376465259849, '用户投资体验感得分': 5, '沪深300指数比较得分': 0, '中证500指数比较得分': 0, '基准比较得分': 0, '同行业股票比较对比得分': 0}
您做的选择数量需要大于5种
None
{'600009.SH': 1}
600009.SH 航空机场 12


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00,  8.21it/s]


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
600009.SH: 近1天年化收益率位于同行业股票Top25%(3/12), 年化波动率位于同行业股票Top25% (3/12)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 9.194793033518906, '用户投资体验感得分': 5, '沪深300指数比较得分': 0, '中证500指数比较得分': 0, '基准比较得分': 0, '同行业股票比较对比得分': 0}
您做的选择数量需要大于5种
None
{'600000.SH': 100}
600000.SH 银行 41


...

------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
600015.SH: 近1天年化收益率位于同行业股票Top60%(25/41), 年化波动率位于同行业股票Top60% (25/41)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 9.413525301733252, '用户投资体验感得分': 5, '沪深300指数比较得分': 0, '中证500指数比较得分': 0, '基准比较得分': 0, '同行业股票比较对比得分': 0}
您做的选择数量需要大于5种
None
{'600703.SH': 1}
600703.SH 光学光电子 87


100%|██████████████████████████████████████████████████████████████████████████████████| 87/87 [00:11<00:00,  7.36it/s]


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
600703.SH: 近1天年化收益率位于同行业股票Top81%(71/87), 年化波动率位于同行业股票Top81% (71/87)。
{'用户收益偏好得分': 13.28283954107404, '用户风险偏好得分': 9.249637446628135, '用户投资体验感得分': 5, '沪深300指数比较得分': 13.98273381094074, '中证500指数比较得分': 13.897114299869612, '基准比较得分': 0, '同行业股票比较对比得分': 59.99972464308864}
[({'select_dict': {'002460.SZ': 3}, 'return_preference': 0, 'seq_len': 90, 'start_date': '20220601', 'end_date': '20220602'}, {'用户收益偏好得分': 0.0, '用户风险偏好得分': 0.9389018653439531, '用户投资体验感得分': 0.0, '沪深300指数比较得分': 0.0, '中证500指数比较得分': 0.0, '基准比较得分': 0.0, '同行业股票比较对比得分': 0.0}), ({'select_dict': {'600009.SH': 1}, 'return_preference': 0, 'seq_len': 90, 'start_date': '20220601', 'end_date': '20220602'}, {'用户收益偏好得分': 0.0, '用户风险偏好得分': 0.9767640430972154, '用户投资体验感得分': 0.0, '沪深300指数比较得分': 0.0, '中证500指数比较得分': 0.0, '基准比较得分': 0.0, '同行业股票比较对比得分': 0.0}), ({'select_dic

100%|████████████████████████████████████████████████████████████████████████████████| 130/130 [00:19<00:00,  6.70it/s]


------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
300072.SZ: 近1天年化收益率位于同行业股票Top82%(107/130), 年化波动率位于同行业股票Top82% (107/130)。
{'用户收益偏好得分': 0, '用户风险偏好得分': 9.456902169318989, '用户投资体验感得分': 5, '沪深300指数比较得分': 0, '中证500指数比较得分': 0, '基准比较得分': 0, '同行业股票比较对比得分': 0}
[({'select_dict': {'002460.SZ': 3}, 'return_preference': 0, 'seq_len': 90, 'start_date': '20220601', 'end_date': '20220602'}, {'用户收益偏好得分': 0.0, '用户风险偏好得分': 0.09928218020379134, '用户投资体验感得分': 0.0, '沪深300指数比较得分': 0.0, '中证500指数比较得分': 0.0, '基准比较得分': 0.0, '同行业股票比较对比得分': 0.0}), ({'select_dict': {'600009.SH': 1}, 'return_preference': 0, 'seq_len': 90, 'start_date': '20220601', 'end_date': '20220602'}, {'用户收益偏好得分': 0.0, '用户风险偏好得分': 0.10328583563718459, '用户投资体验感得分': 0.0, '沪深300指数比较得分': 0.0, '中证500指数比较得分': 0.0, '基准比较得分': 0.0, '同行业股票比较对比得分': 0.0}), ({'select_dict': {'600000.SH': 100}, 'return_preference': 0, 'seq_len': 